In [ ]:
!pip install split-image
!pip install networkx

In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import PIL
import networkx as nx

from IPython.display import HTML
from IPython.display import Image as IPImage
from PIL import Image
from base64 import b64encode
from split_image import split_image
from sklearn.metrics.pairwise import cosine_similarity

# **1. Load dataset**

In [ ]:
main_path = "/kaggle/input/asltest/"
video_input_test = main_path + "zero_test.mp4"
video_input_true = main_path + "zero_true.mp4"

In [ ]:
def play(filename):
    html = ''
    video = open(filename,'rb').read()
    src = 'data:video/mp4;base64,' + b64encode(video).decode()
    html += '<video width=1000 controls autoplay loop><source src="%s" type="video/mp4"></video>' % src 
    return HTML(html)

play(video_input_true)

# **2. Splitting videos**

In [ ]:
vidObj = cv2.VideoCapture(video_input_true)
countTrue = 0

while True: 
    success, image = vidObj.read() 
    if success:
        cv2.imwrite(f"frame_true_{countTrue}.jpg", image) 
    else: 
        break
    countTrue += 1
    
img = cv2.imread("frame_true_0.jpg")
trueHeight = img.shape[0]
trueWidth = img.shape[1]

In [ ]:
vidObj = cv2.VideoCapture(video_input_test)
countTest = 0

while True: 
    success, image = vidObj.read() 
    if success:
        resize = cv2.resize(image, (trueWidth, trueHeight))
        cv2.imwrite(f"frame_test_{countTest}.jpg", resize) 
    else: 
        break
    countTest += 1

In [ ]:
for i in range(0, countTrue, 10):
    img = mpimg.imread(f'./frame_true_{i}.jpg')
    imgplot = plt.imshow(img)
    plt.show()

In [ ]:
for i in range(0, countTest, 10):
    img = mpimg.imread(f'./frame_test_{i}.jpg')
    imgplot = plt.imshow(img)
    plt.show()

# **3. Extracting squares, comparing similarity and constructing graphs**

In [ ]:
rowSquares = 10
columnSquares = 10
totalSquares = rowSquares * columnSquares
delta = 0.5
graphTrue = nx.Graph()

split_image('/kaggle/working/frame_true_0.jpg', rowSquares, columnSquares, True, False)

for i in range(totalSquares):
    image1 = cv2.imread(f'/kaggle/working/frame_true_0_{i}.jpg')
    image1 = image1.transpose(2,0,1).reshape(3,-1)
    image1_norm = cv2.normalize(image1, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    for j in range(i+1, totalSquares):
        image2 = cv2.imread(f'/kaggle/working/frame_true_0_{j}.jpg')
        image2 = image2.transpose(2,0,1).reshape(3,-1)
        image2_norm = cv2.normalize(image2, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        meanCorr = cosine_similarity(image1_norm, image2_norm).mean()
        if meanCorr > delta:
            graphTrue.add_node(i)
            graphTrue.add_node(j)
            graphTrue.add_edge(i, j, weight=meanCorr)

nx.draw(graphTrue, node_color="red", with_labels=True, font_color="white", font_size=10, node_size=500, alpha=0.1)
plt.show()

In [ ]:
graphTest = nx.Graph()

split_image('/kaggle/working/frame_test_0.jpg', rowSquares, columnSquares, True, False)

for i in range(totalSquares):
    image1 = cv2.imread(f'/kaggle/working/frame_test_0_{i}.jpg')
    image1 = image1.transpose(2,0,1).reshape(3,-1)
    image1_norm = cv2.normalize(image1, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    for j in range(i+1, totalSquares):
        image2 = cv2.imread(f'/kaggle/working/frame_test_0_{j}.jpg')
        image2 = image2.transpose(2,0,1).reshape(3,-1)
        image2_norm = cv2.normalize(image2, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        meanCorr = cosine_similarity(image1_norm, image2_norm).mean()
        if meanCorr > delta:
            graphTest.add_node(i)
            graphTest.add_node(j)
            graphTest.add_edge(i, j, weight=meanCorr)

nx.draw(graphTest, node_color="red", with_labels=True, font_color="white", font_size=10, node_size=500, alpha=0.1)
plt.show()

In [ ]:
nx.graph_edit_distance(graphTest, graphTrue)